In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# Beispielbild laden
image_path = "../data/pzl_1_processed/pzl_1_full_front.jpg"  # Neuer Pfad zum Gesamtpuzzle
image = cv2.imread(image_path)
if image is None:
    print(f"Fehler beim Laden des Bildes: {image_path}")
else:
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Originalbild")
    plt.axis("off")
    plt.show()

In [ ]:
## Wandle das Bild in Graustufen um ohne die OpenCV-Funktion cvtColor zu verwenden
gray_image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140])
# Konvertiere das Graustufenbild in uint8
gray_image = np.clip(gray_image, 0, 255).astype(np.uint8)
# Zeige das Graustufenbild an
plt.imshow(gray_image, cmap='gray')
plt.title("Graustufenbild")
plt.axis("off")
plt.show()

### Kantenerkennung mithilfe vom Sobel-Operator

In [ ]:
#Erkenne mithilfe vom Sobel-Operator 
#Kanten im Graustufenbild 
#ohne die OpenCV-Funktion Sobel zu verwenden

def sobel_operator(image):
    # Sobel-Kernel für x- und y-Richtung
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    
    sobel_y = np.array([[1, 2, 1],
                        [0, 0, 0],
                        [-1,-2,-1]])
    
    # Initialisiere das Ergebnisbild
    gradient_magnitude = np.zeros_like(image)

    # Wende den Sobel-Operator an
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            gx = np.sum(sobel_x * image[i-1:i+2, j-1:j+2])
            gy = np.sum(sobel_y * image[i-1:i+2, j-1:j+2])
            gradient_magnitude[i, j] = np.sqrt(gx**2 + gy**2)
    
    return gradient_magnitude
# Wende den Sobel-Operator auf das Graustufenbild an
edges = sobel_operator(gray_image)
# Normalisiere die Kantenstärke auf den Bereich [0, 255]
edges = np.clip(edges, 0, 255).astype(np.uint8)
# Zeige das Ergebnis an
plt.imshow(edges, cmap='gray')

### Kantenerkennung mithilfe von Canny-Kantendetektion

In [ ]:
# Kantenerkennung mithilfe von Canny-Kantendetektion ohne die OpenCV-Funktion Canny zu verwenden
def canny_edge_detection(image, low_threshold, high_threshold):
    # Wende den Sobel-Operator an
    gradient_magnitude = sobel_operator(image)
    
    # Non-maximum Suppression
    suppressed = np.zeros_like(gradient_magnitude)
    angle = np.arctan2(np.gradient(image, axis=0), np.gradient(image, axis=1)) * 180 / np.pi
    angle[angle < 0] += 180
    
    for i in range(1, gradient_magnitude.shape[0] - 1):
        for j in range(1, gradient_magnitude.shape[1] - 1):
            q = 255
            r = 255
            
            # Winkel anpassen
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                q = gradient_magnitude[i, j + 1]
                r = gradient_magnitude[i, j - 1]
            elif (22.5 <= angle[i, j] < 67.5):
                q = gradient_magnitude[i + 1, j - 1]
                r = gradient_magnitude[i - 1, j + 1]
            elif (67.5 <= angle[i, j] < 112.5):
                q = gradient_magnitude[i + 1, j]
                r = gradient_magnitude[i - 1, j]
            elif (112.5 <= angle[i, j] < 157.5):
                q = gradient_magnitude[i - 1, j - 1]
                r = gradient_magnitude[i + 1, j + 1]

            if (gradient_magnitude[i, j] >= q) and (gradient_magnitude[i, j] >= r):
                suppressed[i, j] = gradient_magnitude[i, j]
            else:
                suppressed[i, j] = 0

    # Double Thresholding
    strong_edges = (suppressed >= high_threshold)
    weak_edges = ((suppressed >= low_threshold) & (suppressed < high_threshold))
    
    # Edge Tracking by Hysteresis
    edges_final = np.zeros_like(suppressed)
    edges_final[strong_edges] = 255
    
    for i in range(1, suppressed.shape[0] - 1):
        if weak_edges[i, j]:
            if ((strong_edges[i + 1, j - 1] or strong_edges[i + 1, j] or strong_edges[i + 1, j + 1]) or
                (strong_edges[i, j - 1] or strong_edges[i, j + 1]) or
                (strong_edges[i - 1, j - 1] or strong_edges[i - 1, j] or strong_edges[i - 1, j + 1])):
                edges_final[i, j] = 255
    return edges_final
# Wende die Canny-Kantendetektion an
#high_treshold = 65 bei einzelnem Puzzle

edges_canny = canny_edge_detection(gray_image, low_threshold=100, high_threshold=65)
# Zeige das Ergebnis der Canny-Kantendetektion an
plt.imshow(edges_canny, cmap='gray')
plt.title("Canny-Kantendetektion")
plt.axis("off")
plt.show()

In [ ]:
#Lade alle einzelen Puzzlestuecke aus dem Verzeichnis welche mit front enden
puzzle_dir = Path("../data/pzl_3_processed/")
puzzle_images = list(puzzle_dir.glob("*_front.jpg"))
if not puzzle_images:
    print("Keine Puzzlestücke gefunden.")
else:
    # Zeige die Anzahl der gefundenen Puzzlestücke an
    print(f"Anzahl der gefundenen Puzzlestücke: {len(puzzle_images)}")
    
    # Zeige die ersten 5 Puzzlestücke nebeneinander an
    fig, axes = plt.subplots(1, min(5, len(puzzle_images)), figsize=(15, 5))
    for ax, puzzle_image in zip(axes, puzzle_images[:5]):
        img = cv2.imread(str(puzzle_image))
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.set_title(puzzle_image.name)
        ax.axis("off")
    plt.tight_layout()
    plt.show()


In [ ]:
# Erstelle den Zielordner für die Canny-Bilder
output_dir = Path("../data/pzl_3_processed/canny_images/")
output_dir.mkdir(parents=True, exist_ok=True)

# Wende Canny und Sobel auf jedes Puzzlestück an und speichere die Ergebnisse
for puzzle_image in puzzle_images:
    img = cv2.imread(str(puzzle_image))
    if img is None:
        print(f"Fehler beim Laden des Bildes: {puzzle_image}")
        continue

    # Konvertiere das Bild in Graustufen
    gray_img = np.dot(img[..., :3], [0.2989, 0.5870, 0.1140])
    gray_img = np.clip(gray_img, 0, 255).astype(np.uint8)

    # Wende Sobel- und Canny-Kantendetektion an
    edges_sobel = sobel_operator(gray_img)
    edges_sobel = np.clip(edges_sobel, 0, 255).astype(np.uint8)

    edges_canny = canny_edge_detection(gray_img, low_threshold=100, high_threshold=65)

    # Speichere die Canny-Kantendetektionsergebnisse im Zielordner
    output_file = output_dir / f"{puzzle_image.stem}_canny.jpg"
    cv2.imwrite(str(output_file), edges_canny)

    print(f"Canny-Kantenerkennung gespeichert: {output_file}")

    # Zeige die Ergebnisse an
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(edges_sobel, cmap='gray')
    plt.title(f"Sobel-Kantenerkennung: {puzzle_image.name}")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(edges_canny, cmap='gray')
    plt.title(f"Canny-Kantenerkennung: {puzzle_image.name}")
    plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
def xor_matching(template_edges, full_edges, roi=None, threshold=None):
    """
    Führt XOR-Matching zwischen dem Kantenbild des Puzzleteils und dem Gesamtpuzzle durch.
    Einschränkt die Region und filtert Ergebnisse basierend auf einem Schwellenwert.
    """
    template_h, template_w = template_edges.shape
    full_h, full_w = full_edges.shape
    
    # Falls eine ROI angegeben ist, beschränke das Gesamtbild auf diese Region
    if roi:
        x_start, y_start, x_end, y_end = roi
        full_edges = full_edges[y_start:y_end, x_start:x_end]
        full_h, full_w = full_edges.shape
    
    # Ergebnisbild für die Matching-Werte
    matching_result = np.zeros((full_h - template_h + 1, full_w - template_w + 1), dtype=np.float32)
    
    # Verschiebe das Template pixelweise über das Gesamtbild
    for y in range(full_h - template_h + 1):
        for x in range(full_w - template_w + 1):
            # Ausschneiden des Bereichs im Gesamtbild
            region = full_edges[y:y + template_h, x:x + template_w]
            
            # Berechne XOR zwischen Template und Region
            xor_result = np.bitwise_xor(template_edges, region)
            
            # Summiere die XOR-Werte (je kleiner, desto besser die Übereinstimmung)
            matching_result[y, x] = np.sum(xor_result)
    
    # Falls ein Schwellenwert angegeben ist, filtere die Ergebnisse
    if threshold:
        matching_result[matching_result > threshold] = np.inf
    
    return matching_result

# Beispiel: Einschränkung der Region und Schwellenwert
roi = (100, 100, 500, 500)  # Region of Interest (x_start, y_start, x_end, y_end)
threshold = 1e6  # Schwellenwert für XOR-Werte

# Lade das Gesamtpuzzle und konvertiere es in ein Kantenbild
full_puzzle_edges = canny_edge_detection(full_puzzle_gray, low_threshold=100, high_threshold=65)

# Lade die Puzzlestücke und führe XOR-Matching durch
for puzzle_image_path in puzzle_images:
    puzzle_piece = cv2.imread(str(puzzle_image_path))
    if puzzle_piece is None:
        print(f"Fehler beim Laden des Puzzlestücks: {puzzle_image_path}")
        continue
    
    puzzle_piece_gray = np.dot(puzzle_piece[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
    puzzle_piece_edges = canny_edge_detection(puzzle_piece_gray, low_threshold=100, high_threshold=65)
    
    matching_result = xor_matching(puzzle_piece_edges, full_puzzle_edges, roi=roi, threshold=threshold)
    
    min_val = np.min(matching_result)
    min_loc = np.unravel_index(np.argmin(matching_result), matching_result.shape)
    
    top_left = (min_loc[1] + roi[0], min_loc[0] + roi[1])  # Anpassung der Position basierend auf ROI
    h, w = puzzle_piece_edges.shape
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv2.rectangle(full_puzzle, top_left[::-1], bottom_right[::-1], (0, 255, 0), 2)
    print(f"Puzzlestück {puzzle_image_path.name} gefunden bei Position {top_left[::-1]} mit minimalem XOR-Wert {min_val:.2f}")

# Zeige das Gesamtpuzzle mit den eingezeichneten Positionen
plt.imshow(cv2.cvtColor(full_puzzle, cv2.COLOR_BGR2RGB))
plt.title("Gefundene Puzzlestücke mit eingeschränktem XOR-Matching")
plt.axis("off")
plt.show()

In [ ]:
# Zeichne die Positionen der Puzzlestücke auf dem Gesamtpuzzle
for puzzle_image_path in puzzle_images:
    puzzle_piece = cv2.imread(str(puzzle_image_path))
    puzzle_piece_gray = np.dot(puzzle_piece[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
    puzzle_piece_edges = canny_edge_detection(puzzle_piece_gray, low_threshold=100, high_threshold=65)
    
    matching_result = xor_matching(puzzle_piece_edges, full_puzzle_edges)
    min_val = np.min(matching_result)
    min_loc = np.unravel_index(np.argmin(matching_result), matching_result.shape)
    
    top_left = min_loc
    h, w = puzzle_piece_edges.shape
    bottom_right = (top_left[1] + w, top_left[0] + h)
    cv2.rectangle(full_puzzle, top_left[::-1], bottom_right[::-1], (0, 255, 0), 2)
    print(f"Puzzlestück {puzzle_image_path.name} gefunden bei Position {top_left[::-1]} mit minimalem XOR-Wert {min_val:.2f}")

# Zeige das Gesamtpuzzle mit den eingezeichneten Positionen
plt.imshow(cv2.cvtColor(full_puzzle, cv2.COLOR_BGR2RGB))
plt.title("Gefundene Puzzlestücke mit XOR-Matching")
plt.axis("off")
plt.show()